In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import tensorflow as tf

In [ ]:
df_train = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')

In [ ]:
df_train.head()

In [ ]:
df_train.drop([ 'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
       'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'],axis = 1,inplace = True)

In [ ]:
df_train.head()

In [ ]:
df_train['Id'] = df_train['Id'].apply(lambda x: x + '.jpg')
#df_train.Pawpularity = df_train.Pawpularity.astype('str')
df_train.head()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import MeanAbsoluteError, MeanAbsolutePercentageError

In [ ]:
train_datagen = ImageDataGenerator(
                rescale = 1.0/255.0,
                horizontal_flip = True,
                vertical_flip=True,
                fill_mode = "nearest",
                width_shift_range = 0.2, 
                height_shift_range=0.2,
                rotation_range=30,
)

In [ ]:
train_data = train_datagen.flow_from_dataframe(
            dataframe = df_train,
            directory='../input/petfinder-pawpularity-score/train',
            x_col = "Id",
            y_col = "Pawpularity",
            has_ext=True,
            class_mode="raw",
            batch_size = 16,
            target_size=(224,224),
            shuffle = True,
        )

In [ ]:
import matplotlib.pyplot as plt
for i in range(4):
    img,lab = train_data.next()
    plt.imshow(img[0])
    plt.show()

In [ ]:
class Block(tf.keras.Model):
    def __init__(self, filters, kernel_size, repetitions, pool_size=2, strides=2):
        super(Block, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.repetitions = repetitions
        
        for i in range(self.repetitions):
            vars(self)[f'conv2D_{i}'] = tf.keras.layers.Conv2D(self.filters,self.kernel_size,padding='same',activation='relu')
        self.max_pool = tf.keras.layers.MaxPool2D(pool_size=pool_size,
                                                  strides=strides,
                                                  padding='same')
  
    def call(self, inputs):
        conv2D_0 = vars(self)['conv2D_0']
        x = conv2D_0(inputs)
        for i in range(1,self.repetitions):
            conv2D_i = vars(self)[f'conv2D_{i}']
            x = conv2D_i(x)
        max_pool = self.max_pool(x)
        
        return max_pool

In [ ]:
class VGG(tf.keras.Model):
    def __init__(self):
        super(VGG,self).__init__()
        self.block1 = Block(64,3,2)
        self.block2 = Block(128,3,2)
        self.block3 = Block(256,3,3)
        self.block4 = Block(512,3,3)
        self.block5 = Block(512,3,3)
        self.flatten = tf.keras.layers.Flatten()
        self.layer_dense = tf.keras.layers.Dense(1024,activation = 'relu')
        self.layer_dense = tf.keras.layers.Dense(512,activation = 'relu')
        self.layer_dense = tf.keras.layers.Dense(256,activation = 'relu')
        self.result = tf.keras.layers.Dense(1,activation = 'relu')
        
    def call(self,inputs):
        x = self.block1(inputs)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = self.flatten(x)
        x = self.layer_dense(x)
        return self.result(x)    

In [ ]:
model = VGG()

In [ ]:
def my_rmse(y_true, y_pred):
    error = tf.cast(y_true,dtype = np.float32)  - tf.cast(y_pred,dtype = np.float32)
    sqr_error = tf.keras.backend.square(error)
    mean_sqr_error = tf.keras.backend.mean(sqr_error)
    sqrt_mean_sqr_error = tf.keras.backend.sqrt(mean_sqr_error)
    return sqrt_mean_sqr_error

In [ ]:
optimizer_ = tf.keras.optimizers.Adam(
    learning_rate=2e-4, beta_1=0.9, beta_2=0.999
)

In [ ]:
model.compile(optimizer=optimizer_,loss=my_rmse,metrics=[MeanAbsoluteError(),MeanAbsolutePercentageError()])

In [ ]:
class Callbackss(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs = {}):
        if logs.get('loss') < 19.6:
            self.model.stop_training = True
call = Callbackss()
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=2, verbose=1)

In [ ]:
NUB_TRAIN_STEPS = train_data.n // train_data.batch_size

In [ ]:
hist = model.fit(train_data,epochs = 300,callbacks=[call,reduce_lr],batch_size = 16, steps_per_epoch=NUB_TRAIN_STEPS)

In [ ]:
test_datagen = ImageDataGenerator(
                rescale = 1.0/255.0,
)
df_test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')

In [ ]:
df_test.drop(['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
       'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'],axis = 1,inplace = True)
df_test.Id = df_test.Id.apply(lambda x:x +'.jpg')
df_test.head()

In [ ]:


test_data  = test_datagen.flow_from_dataframe(
            dataframe = df_test,
            directory='../input/petfinder-pawpularity-score/test',
            x_col = "Id",
            shuffle=False,
            class_mode=None,
            target_size=(224,224),
        )

In [ ]:
pred = model.predict(test_data)

In [ ]:
sub = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')
sub['Pawpularity'] = pred

In [ ]:
sub.to_csv('submission.csv',index = False)